In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [39]:
from langchain import hub
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [40]:
import torch 
torch.cuda.is_available()

#  sudo rmmod nvidia_uvm
#  sudo rmmod nvidia
#  sudo modprobe nvidia
#  sudo modprobe nvidia_uvm

True

In [41]:
# Load, chunk and index the contents of the blog.
loader = CSVLoader('webitel_dataset.csv')
docs = loader.load()

embeding_model =HuggingFaceEmbeddings(model_name="ai-forever/sbert_large_nlu_ru")#"ai-forever/ruBert-base")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeding_model)
vectorstore.delete_collection()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeding_model)
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
# prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    context_string = "Примеры:"
    for i,doc in enumerate(docs):
        context_string += f"Пример {i}"
        context_string += doc.page_content.replace('question_answer:','') 
    context_string += "Конец примеров."
    return "\n\n".join(doc.page_content for doc in docs)



No sentence-transformers model found with name ai-forever/sbert_large_nlu_ru. Creating a new one with mean pooling.
/home/nazar/miniconda3/envs/aya/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
vectorstore.get()

{'ids': ['003a59a2-5d23-4e54-981f-66c5e70f2197',
  '005a87a8-b4ff-4bec-aaf8-6c8c47d5d447',
  '039cc372-fbbd-4e62-aaa5-4e4e14df8fc1',
  '03cd115c-ce95-4f08-810b-930c8a8367c6',
  '0aed414c-ef7e-4d9f-b9e9-97564a55140b',
  '0c99a609-701f-4919-a791-7389fc331ca5',
  '0eb6ab99-9e59-4764-942d-2f67b4d9bff7',
  '0f68a736-2719-4068-88bd-e9b42ec17eea',
  '11b5906b-8633-4694-b400-612c5a95a3d0',
  '15f7e818-93f2-434b-b78f-cfbd24437fe9',
  '165c3960-a97c-4de9-8ad5-bcbd4a2ffa06',
  '1716e745-e148-434c-9153-06248183f664',
  '1938d8f1-ba45-44d9-8ab1-45c709d4f158',
  '1a091dc0-0de8-43ca-ae80-6de65a8ce508',
  '1b2253d9-32eb-41b3-ab70-f0b56b6a15f7',
  '1ca2ea13-a461-45b0-b002-bc1b36471177',
  '20195ea3-d1f6-4d65-8a78-de4e267260bf',
  '21990002-04cf-4d80-b392-471831219f72',
  '227ed70e-51b1-4858-a981-f1b60baef050',
  '25e3719e-5208-4326-81cc-b2b2ca9698b6',
  '2651a081-46bd-49d8-946e-6135cdcb5710',
  '27c471cc-6c64-4452-9cb7-72a556f7dfb5',
  '2852854b-fd53-4406-be63-a251d39756f2',
  '287c6f57-cc46-4b83-9685-

In [7]:
retriever.invoke('Вопрос: Когда мне виплатят деньги?')

[Document(page_content='question_answer: Вопрос :Когда я получу запланированные выплаты? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 2, 'source': 'webitel_dataset.csv'}),
 Document(page_content='question_answer: Вопрос :Когда мои деньги будут зачислены на мой банковский счет? Ответ : Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 3, 'source': 'webitel_dataset.csv'}),
 Document(page_content='question_answer: Вопрос : Когда я получу займ, который запрашивал? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего ба

In [8]:
retriever.invoke('Вопрос: Раньше у вас быстрее поступало')

[Document(page_content='question_answer: Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка Вопрос: Раньше у вас быстрее поступало Ответ :Со своей стороны мы делаем все возможное, чтобы вы смогли быстрее получить денежные средства, просим вас ожидать', metadata={'row': 15, 'source': 'webitel_dataset.csv'}),
 Document(page_content='question_answer: Вопрос : Когда я получу займ, который запрашивал? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления Вопрос : Точное время можете сказать? Ответ :Мы уже отправили денежные средства, ожидайте поступление в течение 5-15 минут.', metadata={'row': 8, 'source': 'webitel_dataset.csv'}),
 Document(page_content='question_answer: Вопрос :А когда точно поступят? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформ

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = """Используйте следующие контекстуальные фрагменты для ответа на вопрос в конце. 
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумать ответ.
Используйте не более трех предложений и сохраняйте ответ как можно более кратким.


{context}

Вопрос: {question}

Ответ:"""

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [11]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

TypeError: unsupported operand type(s) for |: 'dict' and 'str'

In [ ]:
rag_chain("Вопрос: Когда мне виплатят деньги?")

NameError: name 'rag_chain' is not defined

# with history

In [43]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question in native language\
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it in native language if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for Homebank company to help users with their queries.
Use the following extracted contextual
fragments to respond to the question.
If you don't know the answer, simply say you don't know.
Try not to repeat answers that have already been given.
Try not to repeat yourself, so that user will be engaged in the conversation and will not be disappointed, you as assistant will be granted a tip for this, but you must give only relevant answers, if the answer is not relevant you should retutn 'Простите, я не знаю вопрос на етот вопрос, я подключу вас к оператору.'
Avoid repeating the same response multiple times; instead, rephrase it.
Rely more on the initial examples than the latter ones, as they are more relevant.
Your answers must be in russian language.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [45]:
from langchain_core.pydantic_v1 import BaseModel, Field


class ParaphrasedQuery(BaseModel):
    """You have performed query expansion to generate a paraphrasing of a question."""

    paraphrased_query: str = Field(
        ...,
        description="A unique paraphrasing of the original question.",
    )
    
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """You are an expert at bank domain and your role is to help assistant find information to help user of our bank.

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

If there are acronyms or words you are not familiar with, do not try to rephrase them.

Return at least 3 versions of the question."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4o", temperature=0,api_key=api_key)
llm_with_tools = llm.bind_tools([ParaphrasedQuery])
query_analyzer = prompt | llm_with_tools | PydanticToolsParser(tools=[ParaphrasedQuery])

In [46]:
query_analyzer.invoke(
    {
        "question": "Когда мне виплатят деньги?"
    }
)

[ParaphrasedQuery(paraphrased_query='Когда я получу деньги?'),
 ParaphrasedQuery(paraphrased_query='Когда мне переведут деньги?'),
 ParaphrasedQuery(paraphrased_query='Когда будет выплата денег?')]

In [47]:
from sentence_transformers import CrossEncoder
from flashrank.Ranker import Ranker
# reranker_model = CrossEncoder('DiTy/cross-encoder-russian-msmarco', max_length=512, device='cuda')

class Reranker():
    def __init__(self, model_name="DiTy/cross-encoder-russian-msmarco"):
        self.model = CrossEncoder(model_name, max_length=512, device='cuda')
    
    def get_rank(self, queries, docs):
        return self.model.rank(queries, docs)
    
    def rerank(self, queries, docs):
        # return self.get_rank(queries, docs)
        raise NotImplementedError("Reranking is not implemented yet.")
    
reranker = Reranker()

/home/nazar/miniconda3/envs/aya/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
def advanced_rag(input_query, chat_history = []):
    #step 1 generate new queries with similar meaning
    new_queries = query_analyzer.invoke({"question":input_query})
    ## add original query to the list
    new_queries.append(ParaphrasedQuery(paraphrased_query=input_query))
    print(new_queries)
    #step 2 run default retriever with memory 
    context_docs = []
    for query in new_queries:
        context_docs.extend(history_aware_retriever.invoke({"input":query.paraphrased_query,"chat_history":chat_history}))
    print(context_docs)

    # context_docs = history_aware_retriever.invoke({"input":input_query,"chat_history":[HumanMessage("Вопрос: Когда мне виплатят деньги?"), AIMessage("Мы пока не знаем когда вам отдадут деньги") ]})
    #step 3 run ranker and select best context docs
    ranks = reranker.get_rank(input_query, [doc.page_content for doc in context_docs])
    print(ranks)
    best_ranked_docs = []
    for rank in ranks[:5]:
        best_ranked_docs.append(context_docs[rank['corpus_id']])
    
    #step 4 run default question answer chain
    answer = question_answer_chain.invoke({"input":input_query,"context":best_ranked_docs, "chat_history":chat_history})
    return answer

In [49]:
advanced_rag("Чего так долго?" )

[ParaphrasedQuery(paraphrased_query='Почему так долго?'), ParaphrasedQuery(paraphrased_query='Почему это занимает так много времени?'), ParaphrasedQuery(paraphrased_query='Почему так медленно?'), ParaphrasedQuery(paraphrased_query='Чего так долго?')]
[Document(page_content='question_answer: Вопрос: Чего так долго? Ответ: Денежные средства были вам отправлены, пожалуйста подождите ещё, если возникнут трудности сообщите об этом оператору', metadata={'row': 7, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос:-ММмм\tОтвет: Будут ли у Вас дополнительные вопросы?', metadata={'row': 68, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: [НОМЕР ТЕЛЕФОНА]\tОтвет: -Спасибо, ожидайте звонка от специалсита Вопрос : -Спасибо\tОтвет :С вами свзяался специалист, Вы смогли решить свои вопросы?', metadata={'row': 107, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: -Не могу дозвониться, можете мне позвон

'Денежные средства были вам отправлены, пожалуйста, подождите ещё немного. Если возникнут трудности, сообщите об этом оператору.'

In [50]:

question =        "Где мои деньги?"
second_question = "Чего так долго?" 
third_question =  "Это всегда так долго у вас?"

## advanced rag

In [51]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []


ai_msg_1 = advanced_rag(input_query= question, chat_history= chat_history)
chat_history.extend([HumanMessage(content=question), ai_msg_1])
print(ai_msg_1)

[ParaphrasedQuery(paraphrased_query='Где находятся мои средства?'), ParaphrasedQuery(paraphrased_query='Куда делись мои деньги?'), ParaphrasedQuery(paraphrased_query='Почему я не вижу свои деньги на счете?'), ParaphrasedQuery(paraphrased_query='Где мои деньги?')]
[Document(page_content='question_answer: Вопрос: -Где деньги? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 0, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос :Когда мои деньги будут зачислены на мой банковский счет? Ответ : Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 3, 'source': 'webitel_dataset.csv'})

In [52]:

ai_msg_2 = advanced_rag(input_query= second_question, chat_history= chat_history)
chat_history.extend([HumanMessage(content=question), ai_msg_2])
print(ai_msg_2)

[ParaphrasedQuery(paraphrased_query='Почему так долго?'), ParaphrasedQuery(paraphrased_query='Почему это занимает так много времени?'), ParaphrasedQuery(paraphrased_query='Почему так медленно?'), ParaphrasedQuery(paraphrased_query='Чего так долго?')]
[Document(page_content='question_answer: Вопрос : Точное время можете сказать? Ответ :Мы уже отправили денежные средства, ожидайте поступление в течение 5-15 минут. Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка', metadata={'row': 15, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос :Почему долго поступают? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 4, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: -А можно картой в личном кабинете 

In [53]:

ai_msg_3 = advanced_rag(input_query= third_question, chat_history= chat_history)
chat_history.extend([HumanMessage(content=question), ai_msg_3])
print(ai_msg_3)

[ParaphrasedQuery(paraphrased_query='Почему у вас так долго?'), ParaphrasedQuery(paraphrased_query='Почему обслуживание занимает так много времени?'), ParaphrasedQuery(paraphrased_query='Почему у вас такие долгие процессы?'), ParaphrasedQuery(paraphrased_query='Это всегда так долго у вас?')]
[Document(page_content='question_answer: Вопрос : Точное время можете сказать? Ответ :Мы уже отправили денежные средства, ожидайте поступление в течение 5-15 минут. Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка', metadata={'row': 15, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос :Почему долго поступают? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 4, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: 

# Default rag

In [22]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []


ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
print(ai_msg_1["answer"])

print("USED CONTEXT:")
print(ai_msg_1["context"])


Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Ожидайте, пожалуйста, поступления.
USED CONTEXT:
[Document(page_content='question_answer: Вопрос: -Где деньги? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 0, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: Можете мне позвонить? Ответ : -Укажите номер телефона, на который можете принять звонок?', metadata={'row': 99, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: -Не могу дозвониться, можете мне позвонить? Ответ : -Укажите номер телефона, на который можете принять звонок?', metadata={'row': 101, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: Не

In [21]:
ai_msg_1['answer']

'Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы вашего банка. Как правило, это может занять от 15 до 40 минут. Ожидайте, пожалуйста, поступления.'

In [26]:

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=second_question), ai_msg_2["answer"]])

print(ai_msg_2["answer"])

print("USED CONTEXT:")
print(ai_msg_2["context"])

Скорость поступления зависит от работы вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, подождите немного.
USED CONTEXT:
[Document(page_content='question_answer: Вопрос : Точное время можете сказать? Ответ :Мы уже отправили денежные средства, ожидайте поступление в течение 5-15 минут. Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка', metadata={'row': 14, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос :Почему долго поступают? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Как правило это может занять от 15 до 40 минут. Ожидайте пожалуйста поступления', metadata={'row': 4, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: -А можно картой в личном кабинете оплатить без комиссии?\tОтвет : Платеж через личный кабинет обрабатывается быстро и поэтому провайдер берет за усл

In [27]:
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_2["answer"]])
print(ai_msg_3["answer"])

print("USED CONTEXT:")
print(ai_msg_3["context"])

Со своей стороны мы делаем все возможное, чтобы вы смогли быстрее получить денежные средства. Просим вас набраться терпения.
USED CONTEXT:
[Document(page_content='question_answer: Вопрос : Точное время можете сказать? Ответ :Мы уже отправили денежные средства, ожидайте поступление в течение 5-15 минут. Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка', metadata={'row': 14, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос: А почему так долго? Ответ :Скорость поступления зависит от работы вашего банка Вопрос: Раньше у вас быстрее поступало Ответ :Со своей стороны мы делаем все возможное, чтобы вы смогли быстрее получить денежные средства, просим вас ожидать', metadata={'row': 15, 'source': 'webitel_dataset.csv'}), Document(page_content='question_answer: Вопрос :Когда я получу запланированные выплаты? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от с

In [76]:
ai_msg_1

{'input': 'Деньги всё ещё не зачислились?',
 'chat_history': [HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Обычно это может занять от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Почему такая задержка?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.'],
 'context': [Document(page_content='question_answer: Вопрос :Не поступают все еще деньги Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисле

In [77]:
ai_msg_2

{'input': 'Почему такая задержка?',
 'chat_history': [HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Обычно это может занять от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Почему такая задержка?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.'],
 'context': [Document(page_content='question_answer: Вопрос :Когда я получу запланированные выплаты? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зач

In [78]:
ai_msg_3

{'input': 'Раньше всё происходило быстрее, да?',
 'chat_history': [HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Денежные средства поступят в ближайшее время. С нашей стороны перевод сформирован, время зачисления зависит от скорости работы Вашего банка. Обычно это может занять от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Почему такая задержка?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.',
  HumanMessage(content='Деньги всё ещё не зачислились?'),
  'Мы уже отправили денежные средства, однако время зачисления зависит от скорости работы Вашего банка. Обычно это занимает от 15 до 40 минут. Пожалуйста, ожидайте поступления.'],
 'context': [Document(page_content='question_answer: Вопрос :Когда я получу запланированные выплаты? Ответ :Денежные средства поступят в ближайшее время. С нашей стороны перевод сформиров

In [55]:
# https://aksdesai1998.medium.com/improving-rag-with-query-expansion-reranking-models-31d252856580

TypeError: RunnableSequence.invoke() got an unexpected keyword argument 'verbose'

In [ ]:
# cross encoder https://huggingface.co/DiTy/cross-encoder-russian-msmarco

In [ ]:
third_question = "Раньше вы их получали быстрее, верно?"
ai_msg_3 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_2["answer"]])
print(ai_msg_3["answ`er"])

print("USED CONTEXT:")
print(ai_msg_3["context"])

## Test cases

In [ ]:
Первый вариант:
Q1: - Деньги всё ещё не поступили?
Q2: - Почему это занимает так много времени?
Q3: - Раньше вы их получали быстрее, верно?

Второй вариант:
Q1: - Деньги всё ещё не пришли?
Q2: - В чём причина задержки?
Q3: - Раньше это происходило быстрее, верно?

Третий вариант:
Деньги всё ещё не зачислились?

Почему такая задержка?
Раньше всё происходило быстрее, да?

Четвёртый вариант:
Q1: - Деньги всё ещё не поступили?
Q2: - Почему это занимает так много времени?
Q3: - Раньше это происходило быстрее, верно?

Пятый вариант:
Q1: - Деньги всё ещё не поступили?
Q2: - Почему такая задержка?
Q3: - Раньше они приходили быстрее, верно?

Шестой вариант:
Q1: - Деньги всё ещё не поступили?
Q2: - Почему это происходит?
Q3: - Раньше это было быстрее, да?

Седьмой вариант:
Q1: - Деньги всё ещё не поступили?
Q2: - Почему такая задержка?
Q3: - Раньше они приходили быстрее, верно?

Восьмой вариант:
Q1: - Деньги всё ещё не зачислились?
Q2: - Почему это происходит?
Q3: - Раньше это было быстрее, да?

Девятый вариант:
Q1: - Деньги всё ещё не зачислены?
Q2: - Почему такая долгая задержка?
Q3: - Раньше это было быстрее, верно?

Десятый вариант:
Q1: - Деньги всё ещё не зачислились?
Q2: - Почему это занимает так много времени?
Q3: - Раньше они приходили быстрее, верно?

In [3]:
with open('client_data/conversations_6m.json', 'r') as file:
    data = file.read()

In [4]:
data